In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim
from torch.cuda.amp import autocast, GradScaler

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
import torch.distributed as dist


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#writer = SummaryWriter(log_dir="/data/runs")

In [3]:
SEED=1

In [4]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [5]:
START_EPOCH = 0

In [6]:
ARCH = 'resnet18'
EPOCHS = 1
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=1028
VAL_BATCH=1028
WORKERS=8
TRAINDIR="/data/imagenet/train"
VALDIR="/data/imagenet/val"
GPU = 0

In [7]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [ ]:
WORLD_SIZE = 2
BACKEND = 'nccl'
URL = 'tcp://35.88.117.239:1234'
RANK = 0

In [ ]:
dist.init_process_group(backend = BACKEND, init_method = URL, world_size = WORLD_SIZE, rank=RANK)

In [8]:
torch.cuda.set_device(GPU)

In [9]:
cudnn.benchmark = True

In [10]:
global_step = 0

In [11]:
scaler = GradScaler()

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()
    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast(dtype=torch.float16):
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        #writer.add_scalar("Loss/train", loss, global_step = global_step)
        #writer.add_scalar("Acc1/train", acc1, global_step = global_step)
        #writer.add_scalar("Acc5/train", acc5, global_step = global_step)
        #global_step += 1

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % PRINT_FREQ == 0:
            progress.display(i)

In [13]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    scaler = GradScaler()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            with autocast(dtype=torch.float16):
                output = model(images)
                loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

In [14]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [16]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [17]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    global global_step
    lr = LR * (0.1 ** (epoch // 3))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    #writer.add_scalar("LR/train", lr, global_step = global_step)   
    

In [18]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [19]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [20]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [21]:
IMG_SIZE = 224

In [22]:
NUM_CLASSES = 1000

In [23]:
model = models.__dict__[ARCH]()

In [24]:
inf = model.fc.in_features

In [25]:
model.fc.in_features

512

In [26]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [27]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
criterion = nn.CrossEntropyLoss().cuda(0)

In [29]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [30]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [31]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [32]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [33]:
transform_val = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [34]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [35]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [36]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None) 

In [37]:
best_acc1 = 0

In [38]:
allocated_memory = torch.cuda.max_memory_allocated(GPU)
total_memory = torch.cuda.get_device_properties(GPU).total_memory
print(f"Batch size: {TRAIN_BATCH}, allocated memory: {allocated_memory / 1024 / 1024} MB total memory: {total_memory / 1024 / 1024} MB")

Batch size: 1028, allocated memory: 44.6904296875 MB total memory: 14971.875 MB


In [39]:
%%time
global_step = 0
for epoch in range(START_EPOCH, EPOCHS):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)
    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)
           
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][   0/1247]	Time 30.835 (30.835)	Data 10.403 (10.403)	Loss 6.9843e+00 (6.9843e+00)	Acc@1   0.19 (  0.19)	Acc@5   0.49 (  0.49)
Epoch: [0][  10/1247]	Time  1.282 ( 4.018)	Data  0.001 ( 0.946)	Loss 6.9467e+00 (6.9683e+00)	Acc@1   0.29 (  0.15)	Acc@5   1.46 (  0.70)
Epoch: [0][  20/1247]	Time  1.282 ( 2.718)	Data  0.001 ( 0.496)	Loss 6.9173e+00 (6.9504e+00)	Acc@1   0.19 (  0.18)	Acc@5   1.36 (  0.99)
Epoch: [0][  30/1247]	Time  1.291 ( 2.259)	Data  0.002 ( 0.337)	Loss 6.8451e+00 (6.9252e+00)	Acc@1   0.58 (  0.27)	Acc@5   1.36 (  1.16)
Epoch: [0][  40/1247]	Time  1.292 ( 2.025)	Data  0.001 ( 0.255)	Loss 6.7614e+00 (6.8917e+00)	Acc@1   0.68 (  0.33)	Acc@5   2.53 (  1.38)
Epoch: [0][  50/1247]	Time  1.308 ( 1.883)	Data  0.001 ( 0.205)	Loss 6.6512e+00 (6.8522e+00)	Acc@1   0.88 (  0.41)	Acc@5   2.33 (  1.61)
Epoch: [0][  60/1247]	Time  1.306 ( 1.789)	Data  0.001 ( 0.172)	Loss 6.5697e+00 (6.8139e+00)	Acc@1   0.78 (  0.47)	Acc@5   3.40 (  1.83)
Epoch: [0][  70/1247]	Time  1.309 ( 1.721